In [1]:
# Import Libraries
import psycopg2
import configparser
import pandas as pd

%load_ext sql 

## Extract

In [2]:
# read data 
data = pd.read_excel('Material_Groups_Mapping_Table (3).xlsx')
data.head(3)

,Material Group,Material Group Desc.,Spend Group,Excluded in Analysis,Spend Category,Spend Subcategory
0,01,Material group 1,NaN,Yes,NaN,NaN
1,02,Material group 2,NaN,Yes,NaN,NaN
2,AMENITIES,Sundries,Staff Related,No,Sundries,Consumables Sundries


In [3]:
# read data
raw = pd.read_excel('me2n_purchasing_documents (3).xlsx')
data2 = raw.copy()
data2.head(3)

,Purchasing Document,Item,Purchasing Doc. Type,Supplier/Supplying Plant,Material,Short Text,Material Group,Document Date,Collective Number,Created by,Price in Currency Conv,Currency
0,4300000004,10,ZCO,101707 MEDIA MARKETING,NaN,MAINTENANCE OF PAASENNGER LIFT,OFCBLDG,2020-02-21,NaN,ABDRAHIM,23516.0,MYR
1,4300000005,10,ZCO,101388 PRO TECHNOLOGY SDN BHD,NaN,RENTAL OF 19 BYD ELEC. FORKLIFT,SER.CON,2020-02-21,NaN,ABDRAHIM,68400.0,MYR
2,4300000006,10,ZCO,101022 LA ENGINEERING SDN BHD,NaN,RENTAL OF 17 UNIT FORKLIFT,SER.CON,2020-02-21,NaN,ABDRAHIM,176400.0,MYR


## Transform

In [4]:
def clean_map(df):
    # remove Yes - excluded in analysis
    # remove nan  
    clean_df = df.drop(df[df['Excluded in Analysis'] == 'Yes'].index).dropna()
    return clean_df

In [5]:
clean_map = clean_map(data)
clean_map.shape

(155, 6)

In [6]:
def clean_transaction(df):
    # change purchasing from int to str
    # make new month column
    # drop collective number column 
    # drop material 
    df['Purchasing Document'] = df['Purchasing Document'].astype(str)
    df['Month'] = df['Document Date'].dt.strftime('%b')
    df.drop(columns=['Collective Number', 'Material'], inplace=True)
    return df

In [7]:
clean_trans = clean_transaction(data2)
clean_trans.head()

,Purchasing Document,Item,Purchasing Doc. Type,Supplier/Supplying Plant,Short Text,Material Group,Document Date,Created by,Price in Currency Conv,Currency,Month
0,4300000004,10,ZCO,101707 MEDIA MARKETING,MAINTENANCE OF PAASENNGER LIFT,OFCBLDG,2020-02-21,ABDRAHIM,23516.0,MYR,Feb
1,4300000005,10,ZCO,101388 PRO TECHNOLOGY SDN BHD,RENTAL OF 19 BYD ELEC. FORKLIFT,SER.CON,2020-02-21,ABDRAHIM,68400.0,MYR,Feb
2,4300000006,10,ZCO,101022 LA ENGINEERING SDN BHD,RENTAL OF 17 UNIT FORKLIFT,SER.CON,2020-02-21,ABDRAHIM,176400.0,MYR,Feb
3,4300000007,10,ZCO,101613 FASHION HOUSE,INDUSTRIAL COARSE SALT,CHEMICAL,2020-06-03,SHANIZA,157500.0,MYR,Jun
4,4500000254,10,ZNB,103230 NASIONAL BERHAD,MODEL LORI BERSKALA 1/64 KONTENA/ TANKER,MARKETING,2020-01-09,ABDRAHIM,7000.0,MYR,Jan


## Load

#### Set up Connection and Database

In [8]:
# Read parameters from the config file
config = configparser.ConfigParser()
config.read('config.cfg')
DB_NAME_DEFAULT = config.get('SQL', 'DB_NAME_DEFAULT')
DB_USER = config.get('SQL', 'DB_USER')
DB_PASSWORD = config.get('SQL', 'DB_PASSWORD')

In [9]:
# Connect to default database
conn = psycopg2.connect("host=127.0.0.1 dbname={} user={} password={}".format(DB_NAME_DEFAULT, DB_USER, DB_PASSWORD))
conn.set_session(autocommit=True)
cur = conn.cursor()

In [10]:
# Create Database
cur.execute("CREATE DATABASE reports WITH ENCODING 'utf8' TEMPLATE template0")

DuplicateDatabase: database "reports" already exists


In [11]:
# Close Connection to default database
conn.close()

In [31]:
# Connect to reports database
conn = psycopg2.connect('host=127.0.0.1 dbname=reports user={} password={}'.format(DB_USER, DB_PASSWORD))
cur = conn.cursor()

#### Set up table for staging -  storage for raw data

In [13]:
cur. execute(

'''
CREATE TABLE IF NOT EXISTS staging_table (
    "Purchasing Document" VARCHAR(50), 
    "Item" INTEGER, 
    "Purchasing Doc. Type" VARCHAR(10), 
    "Supplier/Supplying Plant" VARCHAR(50),
    "Material" FLOAT NULL, 
    "Short Text" VARCHAR(100),
    "Material Group" VARCHAR(50),
    "Document Date" DATE,
    "Collective Number" VARCHAR(30) NULL, 
    "Created by" VARCHAR(50),
    "Price in Currency Conv" FLOAT,
    "Currency" VARCHAR(30),
    "Month" VARCHAR(5)
)
'''
            )
conn.commit()

In [14]:
## insert csv to staging table
for i, row in raw.iterrows():
    try: 
        cur.execute(
        '''
        INSERT INTO staging_table
        ("Purchasing Document", "Item", "Purchasing Doc. Type",
        "Supplier/Supplying Plant", "Material", "Short Text",
        "Material Group", "Document Date", "Collective Number", "Created by",
         "Price in Currency Conv", "Currency")
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        ''', row.tolist())
    
        conn.commit()
    
    except psycopg2.DataError as e:
        # Print the problematic row causing the error
        print("Error inserting row:", row.tolist())
        print("Error message:", e)
        # Rollback the transaction to continue with the next row
        conn.rollback()

#### Set up mapping table

In [15]:
cur. execute(

'''
CREATE TABLE IF NOT EXISTS mapping_table (
    "Material Group" VARCHAR(50) PRIMARY KEY,
    "Material Group Desc" VARCHAR(50),
    "Spend Group" VARCHAR(50),
    "Excluded in Analysis" VARCHAR(5),
    "Spend Category" VARCHAR(50),
    "Spend Subcategory" VARCHAR(50)
)
'''
            )
conn.commit()

In [16]:
## insert csv to mapping_table
for i, row in clean_map.iterrows():
    try: 
        cur.execute(
        '''
        INSERT INTO mapping_table
        ("Material Group", "Material Group Desc", "Spend Group", "Excluded in Analysis", "Spend Category", "Spend Subcategory")
        VALUES (%s, %s, %s, %s, %s, %s)
        ''', row.tolist())
    
        conn.commit()
    
    except psycopg2.DataError as e:
        # Print the problematic row causing the error
        print("Error inserting row:", row.tolist())
        print("Error message:", e)
        # Rollback the transaction to continue with the next row
        conn.rollback()

#### Set up table for storage of transactional data (clean data) 

In [17]:
cur. execute(

'''
CREATE TABLE IF NOT EXISTS transaction_table (
    "Purchasing Document" VARCHAR(50), 
    "Item" INTEGER, 
    "Purchasing Doc. Type" VARCHAR(10), 
    "Supplier/Supplying Plant" VARCHAR(50),  
    "Short Text" VARCHAR(100),
    "Material Group" VARCHAR(50),
    "Document Date" DATE, 
    "Created by" VARCHAR(50),
    "Price in Currency Conv" INTEGER,
    "Currency" VARCHAR(20),
    "Month" VARCHAR(5),
    FOREIGN KEY ("Material Group") REFERENCES mapping_table("Material Group")
)
'''
            )
conn.commit()

In [18]:
## insert csv to transaction table 
for i, row in clean_trans.iterrows():
    try: 
        cur.execute(
        '''
        INSERT INTO transaction_table
        ("Purchasing Document", "Item", "Purchasing Doc. Type",
        "Supplier/Supplying Plant", "Short Text", 
        "Material Group", "Document Date", "Created by", 
        "Price in Currency Conv", "Currency", "Month")
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        ''', row.tolist())
    
        conn.commit()
    
    except psycopg2.DataError as e:
        # Print the problematic row causing the error
        print("Error inserting row:", row.tolist())
        print("Error message:", e)
        # Rollback the transaction to continue with the next row
        conn.rollback()

### Query - Postgres(SQL) 

In [32]:
# Connecting to coffeeshops database
conn_string = "postgresql://{}:{}@127.0.0.1/reports".format(DB_USER, DB_PASSWORD)
%sql $conn_string

#### 1. What is the top 10 highest transaction of supplier/supplying plant and which group?

In [34]:
# Create a cursor object
conn = psycopg2.connect('host=127.0.0.1 dbname=reports user={} password={}'.format(DB_USER, DB_PASSWORD))
cur = conn.cursor()

# Define your SQL query
sql_query = """
SELECT "Price in Currency Conv", "Supplier/Supplying Plant", "Material Group" 
FROM transaction_table
ORDER BY "Price in Currency Conv" DESC
LIMIT 10;
"""

# Execute the SQL query
cur.execute(sql_query)

# Fetch all rows from the result set
rows = cur.fetchall()

# Print the results
for row in rows:
    print(row) 

# Close the cursor and connection
cur.close()
conn.close()

(3230250, '101022     LA ENGINEERING SDN BHD', 'MACHINERY')
(3230250, '101022     LA ENGINEERING SDN BHD', 'MACHINERY')
(2478000, '101022     LA ENGINEERING SDN BHD', 'MACHINERY')
(2478000, '101022     LA ENGINEERING SDN BHD', 'MACHINERY')
(2323000, '100865     PROTECTIVE SDN BHD', 'MACHINERY')
(2286463, '100865     PROTECTIVE SDN BHD', 'MACHINERY')
(883439, '103094     KIYO SDN BHD', 'IT.SWARE')
(864000, '101942    GIFT & PREMIUM SDN BHD', 'IT.SWARE')
(698400, '100303     PLASTICS SDN BHD', 'PCK.SBAG')
(560000, '103094     KIYO SDN BHD', 'CHEMICAL')


#### 2. What is the expenditure breakdown by material group?

In [35]:
# Create a cursor object
conn = psycopg2.connect('host=127.0.0.1 dbname=reports user={} password={}'.format(DB_USER, DB_PASSWORD))
cur = conn.cursor()

# Define your SQL query
sql_query = """
SELECT "Material Group", SUM("Price in Currency Conv") AS "total_expenditure"
FROM transaction_table
GROUP BY "Material Group"
ORDER BY "total_expenditure" DESC
LIMIT 10;
"""

# Execute the SQL query
cur.execute(sql_query)

# Fetch all rows from the result set
rows = cur.fetchall()

# Print the results
for row in rows:
    print(row) 

# Close the cursor and connection
cur.close()
conn.close()

('MACHINERY', 25588746)
('PCK.SBAG', 6457670)
('CHEMICAL', 4646147)
('SER.CON', 3454423)
('PCK.MBAG', 2361530)
('SUNDRIES', 2222361)
('IT.SWARE', 2159026)
('PCK.LBAG', 1369185)
('MARKETING', 1276989)
('INST', 788954)


#### 3. What is the total expenditure each month?

In [38]:
# Create a cursor object
conn = psycopg2.connect('host=127.0.0.1 dbname=reports user={} password={}'.format(DB_USER, DB_PASSWORD))
cur = conn.cursor()

# Define your SQL query
sql_query = """
SELECT "Month", SUM("Price in Currency Conv") AS "total_expenditure"
FROM transaction_table
GROUP BY "Month"
ORDER BY 
	CASE "Month"
		WHEN 'Jan' THEN 1
		WHEN 'Feb' THEN 2
		WHEN 'Mar' THEN 3
		WHEN 'Apr' THEN 4
		WHEN 'May' THEN 5
		WHEN 'Jun' THEN 6
		WHEN 'Jul' THEN 7
		WHEN 'Aug' THEN 8
		WHEN 'Sep' THEN 9
		WHEN 'Oct' THEN 10
    	WHEN 'Nov' THEN 11
   		WHEN 'Dec' THEN 12
	END ASC;
"""

# Execute the SQL query
cur.execute(sql_query)

# Fetch all rows from the result set
rows = cur.fetchall()

# Print the results
for row in rows:
    print(row) 

# Close the cursor and connection
cur.close()
conn.close()

('Jan', 2837241)
('Feb', 17545406)
('Mar', 10237264)
('Apr', 13440465)
('May', 7515179)
('Jun', 3554472)


#### 4. Which suppliers are considered regular suppliers?

In [39]:
# Create a cursor object
conn = psycopg2.connect('host=127.0.0.1 dbname=reports user={} password={}'.format(DB_USER, DB_PASSWORD))
cur = conn.cursor()

# Define your SQL query
sql_query = """
SELECT "Supplier/Supplying Plant", COUNT(*) AS "Frequency"
FROM transaction_table
GROUP BY "Supplier/Supplying Plant"
ORDER BY "Frequency" DESC
LIMIT 20;

"""

# Execute the SQL query
cur.execute(sql_query)

# Fetch all rows from the result set
rows = cur.fetchall()

# Print the results
for row in rows:
    print(row) 

# Close the cursor and connection
cur.close()
conn.close()

('101613     FASHION HOUSE', 464)
('101022     LA ENGINEERING SDN BHD', 331)
('101455     ROTO PAPER SDN BHD', 317)
('101942    GIFT & PREMIUM SDN BHD', 298)
('100198     SOLUTIONS (M) SDN BHD', 297)
('100684     FLOTECH SDN BHD', 274)
('100865     PROTECTIVE SDN BHD', 264)
('101707     MEDIA MARKETING', 245)
('101182     MUL GALY MARKETING', 228)
('100828     HY SDN BHD', 224)
('101388     PRO TECHNOLOGY SDN BHD', 202)
('103157     NETWORK', 124)
('100339     CZO SDN BHD', 109)
('100047     ENGINEERING HARDWARE', 94)
('100303     PLASTICS SDN BHD', 82)
('103094     KIYO SDN BHD', 75)
('100569     MARKETING SDN BHD', 62)
('100196     AVEREX SDN BHD', 45)
('103230     NASIONAL BERHAD', 40)
('100154     ARCA SDN BHD', 26)


#### 5. Create new tables that joins the dimension table  

In [46]:
# Create a cursor object
conn = psycopg2.connect('host=127.0.0.1 dbname=reports user={} password={}'.format(DB_USER, DB_PASSWORD))
cur = conn.cursor()

# Define your SQL query
sql_query = """
CREATE TABLE tran_map_table AS
SELECT 
	transaction_table.*,
	mapping_table."Material Group Desc",
    mapping_table."Spend Group",
    mapping_table."Excluded in Analysis",
    mapping_table."Spend Category",
    mapping_table."Spend Subcategory"
FROM transaction_table
INNER JOIN mapping_table ON transaction_table."Material Group" = mapping_table."Material Group";
"""

# Execute the SQL query
cur.execute(sql_query)

# Fetch all rows from the result set
rows = cur.fetchall()

# Print the results
for row in rows:
    print(row) 

# Close the cursor and connection
cur.close()
conn.close()

DuplicateTable: relation "tran_map_table" already exists


#### 6. What is the expenditure cost for each category?

In [48]:
# Create a cursor object
conn = psycopg2.connect('host=127.0.0.1 dbname=reports user={} password={}'.format(DB_USER, DB_PASSWORD))
cur = conn.cursor()

# Define your SQL query
sql_query = """
SELECT "Spend Category",
	SUM("Price in Currency Conv") AS "Total Spend (RM)",
	Count("Spend Category") AS "Frequency"
FROM tran_map_table
GROUP BY "Spend Category"
ORDER BY "Total Spend (RM)" DESC
LIMIT 20;
"""

# Execute the SQL query
cur.execute(sql_query)

# Fetch all rows from the result set
rows = cur.fetchall()

# Print the results
for row in rows:
    print(row) 

# Close the cursor and connection
cur.close()
conn.close()

('Machinery', 25588746, 896)
('Packaging Materials', 10397185, 275)
('Chemicals', 4646147, 95)
('Service Contract', 3454423, 116)
('IT Related ', 2591045, 48)
('Sundries', 2314319, 241)
('Advertising', 1276989, 85)
('Maintenance Services', 1130408, 193)
('Instrument', 788954, 131)
('Spare Parts', 778216, 495)
('Packaging Pallets', 703358, 8)
('Staff Amenities', 542606, 549)
('Lab Expenses', 305855, 69)
('Hardware', 300846, 282)
('Lubricant', 120983, 20)
('Maintenance Vehicles', 102805, 25)
('Stationery', 61452, 332)
('Warehouse', 13300, 2)
('Upkeep Office', 6800, 4)
('Gift', 3740, 1)


#### 7. What suppliers are engaged for machinery category and the expenses breakdown for each supplier?

In [49]:
# Create a cursor object
conn = psycopg2.connect('host=127.0.0.1 dbname=reports user={} password={}'.format(DB_USER, DB_PASSWORD))
cur = conn.cursor()

# Define your SQL query
sql_query = """
SELECT "Supplier/Supplying Plant", SUM("Price in Currency Conv") AS "Total Expenditure Machinery"
FROM tran_map_table
WHERE "Spend Category" = 'Machinery'
GROUP BY "Supplier/Supplying Plant"
ORDER BY "Total Expenditure Machinery" DESC;
"""

# Execute the SQL query
cur.execute(sql_query)

# Fetch all rows from the result set
rows = cur.fetchall()

# Print the results
for row in rows:
    print(row) 

# Close the cursor and connection
cur.close()
conn.close()

('101022     LA ENGINEERING SDN BHD', 14484729)
('100865     PROTECTIVE SDN BHD', 5898875)
('101942    GIFT & PREMIUM SDN BHD', 1650957)
('101455     ROTO PAPER SDN BHD', 1013954)
('101707     MEDIA MARKETING', 773048)
('101388     PRO TECHNOLOGY SDN BHD', 240260)
('101613     FASHION HOUSE', 239833)
('100684     FLOTECH SDN BHD', 219485)
('100828     HY SDN BHD', 193600)
('101182     MUL GALY MARKETING', 193110)
('100047     ENGINEERING HARDWARE', 135518)
('103230     NASIONAL BERHAD', 107979)
('100339     CZO SDN BHD', 74560)
('103157     NETWORK', 69452)
('103094     KIYO SDN BHD', 51530)
('100196     AVEREX SDN BHD', 43982)
('100095     FLUID SYSTEMS SDN BHD', 43884)
('100006     BULK TRADING SDN BHD', 34761)
('100569     MARKETING SDN BHD', 29662)
('100154     ARCA SDN BHD', 27380)
('100026     TECHNOLOGIES SDN BHD', 27015)
('100303     PLASTICS SDN BHD', 17987)
('100198     SOLUTIONS (M) SDN BHD', 17185)
